In [1]:
import pandas as pd
import numpy as np
import statistics
from statistics import mode
import re
import sys
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/clean_data/IPIP_300_AN.csv")

In [3]:
df.head(3)

,item#,Item,Sign,Key,Dimension,Facet,f_dimension,f_facet
0,i1,Worry about things.,+N1,N1,Neuroticism,Anxiety,dimension1,dimension1_facet7
1,i10,Like order.,+C2,C2,Conscientiousness,Orderliness,dimension3,dimension3_facet3
2,i100,Love order and regularity.,+C2,C2,Conscientiousness,Orderliness,dimension3,dimension3_facet3


In [4]:
df.iloc[2]['f_dimension']

'dimension3'

In [5]:
def mod_dim(dim):
    lis = [df.iloc[i]['Dimension'] for i,e in df.iterrows() if df.iloc[i]['f_dimension'] == f'{dim}']
    try:
        return mode(lis)
    except:
        return f"Dimension not accurate enough"
    
def mod_fac(dim):
    lis = [df.iloc[i]['Facet'] for i,e in df.iterrows() if df.iloc[i]['f_facet'] == f'{dim}']
    try:
        return mode(lis)
    except:
        return f"Facet not accurate enough"

In [6]:
df['fa_dim'] = df['f_dimension'].apply(mod_dim)
df['fa_facet'] = df['f_facet'].apply(mod_fac)

In [7]:
df.drop(columns = ['f_dimension', 'f_facet'], inplace=True)
df.head(3)

,item#,Item,Sign,Key,Dimension,Facet,f_facet,fa_dim,fa_facet
0,i1,Worry about things.,+N1,N1,Neuroticism,Anxiety,dimension1_facet7,Neuroticism,Anxiety
1,i10,Like order.,+C2,C2,Conscientiousness,Orderliness,dimension3_facet3,Conscientiousness,Orderliness
2,i100,Love order and regularity.,+C2,C2,Conscientiousness,Orderliness,dimension3_facet3,Conscientiousness,Orderliness
